In [2]:
from selenium import webdriver

### selenium
- http://www.seleniumhq.org/about/platforms.jsp
- 기본 사용법
- 파일 업로드
    - google vision api 웹페이지에 이미지 파일을 업로드, 결과 데이터를 수집
- 웹데이터 스크래핑
    - gmarket 페이지 베스트셀러 200 상품 수집
    - 네이버 본인 계정의 메일 수집
- 자동화
    - 특정 워드프레스 사이트의 게시물에 댓글을 자동으로 달기
- 이미지 스크린샷
    - youtube 영상 이미지를 크롤링 
    - pillow : Python에서 이미지 처리에 관련된 패키지

### 셀레니움 기본 사용법

In [4]:
# 브라우저 열기 - driver object 와 커넥이 되어 있다.
driver = webdriver.Chrome()

In [5]:
#페이지 이동
driver.get('http://naver.com')

In [10]:
#윈도우 사이즈 조절
driver.set_window_size(600,600)# row * col

In [12]:
#브라주어의 스크롤 조절
script = "window.scrollTo(600,300);"
driver.execute_script(script)

In [13]:
#윈도우 객체 저장
main_window = driver.current_window_handle
main_window

'CDwindow-4712406C16C43B6250858A468AEED3D4'

In [15]:
#윈도우 탭 열기
script = "window.open('https://www.google.com');"
driver.execute_script(script)

In [16]:
driver.current_window_handle

'CDwindow-4712406C16C43B6250858A468AEED3D4'

In [17]:
#현재 열려있는 윈도우 객체 확인
windows = driver.window_handles
windows#현재 열려있는 윈도우가 리스트 형태로 저장되어 있다.
# windows[1] -- google
# windows[0] -- naver

['CDwindow-4712406C16C43B6250858A468AEED3D4',
 'CDwindow-552B77A16D3D5AAD502AB53D8F6FD784']

In [18]:
#현재 윈도우 핸들 변경
driver.switch_to_window(windows[1])

In [20]:
driver.current_window_handle
#현재 포커싱되어 있는 윈도우가 naver -> google 변경되었다.

'CDwindow-552B77A16D3D5AAD502AB53D8F6FD784'

In [22]:
driver.switch_to_window(main_window)
# 다시 메인 윈도우를 naver로 바꿨다.

In [29]:
#페이지 새로고침
script = 'location.reload();'
driver.execute_script(script)

In [39]:
#alert message 처리
script = "alert('selenium test')"
driver.execute_script(script)

In [40]:
alert = driver.switch_to_alert()
alert.text

'selenium test'

In [41]:
alert.accept()

In [43]:
script = "confirm('dss8')"
driver.execute_script(script)

In [44]:
confirm = driver.switch_to_alert()
confirm.text

'dss8'

In [ ]:
confirm.accept() #ok button

In [46]:
confirm.dismiss() #cancel button

In [47]:
#input 창에 문자열 입력하기
driver.find_element_by_css_selector('#query').send_keys('fast campus')

In [48]:
#클릭 이벤트 발생
driver.find_element_by_css_selector('#search_btn').click() 

In [ ]:
#창 닫기
driver.close()

In [52]:
#브라우저 닫기
driver.quit()

### 파일 업로드
- google vision api 페이지에 이미지 파일을 업로드
- iframe 으로 이동

In [54]:
# google vision api 로 이동
url = 'https://cloud.google.com/vision/'
driver = webdriver.Chrome()
driver.get(url)

In [55]:
#iframe으로 이동
iframe = driver.find_element_by_css_selector('#vision_demo_section>iframe')
driver.switch_to_frame(iframe)

iframe 안에 있는 내용을 크롤링해주기 위해서는 switch를 해주어야 합니다.

In [ ]:
# 메임 프레임으로 돌아가기
# driver.switch_to_default_content()

In [57]:
#파일 업로드
#파일 경로는 절대경로로 작성해주셔야 합니다.
#윈도우의 경우 경로 구분자가 \\로 작성되어야 합니다.
file_path = '/home/hskimim/Downloads/dog.png'
driver.find_element_by_css_selector('#input').send_keys(file_path)

In [58]:
driver.find_element_by_css_selector('#webDetection').click()

In [1]:
web_results = driver.find_elements_by_css_selector('#card .container > .row')[:15]
for web_result in web_results:
    print(
    web_result.find_element_by_css_selector('.name').text,
    web_result.find_element_by_css_selector('.score').text
    )
web_results

NameError: name 'driver' is not defined

In [72]:
driver.find_element_by_css_selector('#fullTextAnnotation').click()

In [94]:
doc_results = driver.find_elements_by_css_selector('.page > .container.style-scope.vs-document > fieldset')
len(doc_results)

2

In [96]:
documents = []
for doc_result in doc_results:
    words = doc_result.find_elements_by_css_selector('.container .word')
    sentence_list = []
    for word in words:
        word = word.text.replace(' ','')
        sentence_list.append(word)
    documents.append(' '.join(sentence_list))
documents

['In the dog house', "' 인기를 잃다 , 면목을 잃다"]

In [97]:
driver.quit()

### 스크래핑
- gmarket 베스트 셀러 데이터를 수집

In [98]:
url = 'http://corners.gmarket.co.kr/Bestsellers'
driver = webdriver.Chrome()
driver.get(url)

In [105]:
items = driver.find_elements_by_css_selector('.best-list:nth-child(3) > ul >li')
len(items)

200

In [126]:
dict_list = []
for item in items[:200]:
    dict_list.append({
        'title': item.find_element_by_css_selector('.itemname').text,
        's-price':item.find_element_by_css_selector('.s-price').text\
        .split(" ")[0].replace(',',"")[:-1],
        'O-price':item.find_element_by_css_selector('.o-price').text\
        .split(" ")[0].replace(',',"")[:-1],
        'link':item.find_element_by_css_selector('.itemname').get_attribute('href'),
    })
df = pd.DataFrame(dict_list)
df

,O-price,link,s-price,title
0,20000,http://item.gmarket.co.kr/Item?goodscode=13005...,12900,[하림]하림 프로라인 치킨너겟 1kg 2봉+치킨너겟 100g(증정)
1,19500,http://item.gmarket.co.kr/Item?goodscode=62724...,5900,더다다/원피스형 롱티/신상/루즈/박시/빅사이즈
2,27100,http://item.gmarket.co.kr/Item?goodscode=91202...,16900,CJ 햇반210gx24개/오뚜기밥24개/컵밥10개/컵반
3,65900,http://item.gmarket.co.kr/Item?goodscode=94765...,19900,티셔츠5장9900 원피스/티셔츠/린넨팬츠 세트
4,29000,http://item.gmarket.co.kr/Item?goodscode=14298...,14900,나이키 스우시 라운드 티셔츠
5,100000,http://item.gmarket.co.kr/Item?goodscode=14413...,49900,[풍년]PN풍년 럭키 반값박스 (압력솥+프라이팬+냄비+사은품)
6,32900,http://item.gmarket.co.kr/Item?goodscode=85712...,9900,[영실업]포터리스쿨 도자기스튜디오 LOL서프라이즈 주얼젤리
7,14900,http://item.gmarket.co.kr/Item?goodscode=14260...,4500,[티비제이]TBJ 남녀 여름 티셔츠/팬츠 특가 상품 모음
8,32900,http://item.gmarket.co.kr/Item?goodscode=94895...,9900,올리(Ollie) 여성 여름 샌들/슬리퍼/에스파듀/통굽
9,30000,http://item.gmarket.co.kr/Item?goodscode=63617...,12900,[동원샘물미네마인]동원 샘물 2Lx9병x3팩(총 27병) /생수


In [127]:
is_nodata = df['O-price']==""
df['O-price'][is_nodata]

19     
28     
29     
63     
91     
94     
101    
120    
179    
187    
199    
Name: O-price, dtype: object

In [129]:
df['O-price'][is_nodata] = df['s-price'][is_nodata]
df

,O-price,link,s-price,title
0,20000,http://item.gmarket.co.kr/Item?goodscode=13005...,12900,[하림]하림 프로라인 치킨너겟 1kg 2봉+치킨너겟 100g(증정)
1,19500,http://item.gmarket.co.kr/Item?goodscode=62724...,5900,더다다/원피스형 롱티/신상/루즈/박시/빅사이즈
2,27100,http://item.gmarket.co.kr/Item?goodscode=91202...,16900,CJ 햇반210gx24개/오뚜기밥24개/컵밥10개/컵반
3,65900,http://item.gmarket.co.kr/Item?goodscode=94765...,19900,티셔츠5장9900 원피스/티셔츠/린넨팬츠 세트
4,29000,http://item.gmarket.co.kr/Item?goodscode=14298...,14900,나이키 스우시 라운드 티셔츠
5,100000,http://item.gmarket.co.kr/Item?goodscode=14413...,49900,[풍년]PN풍년 럭키 반값박스 (압력솥+프라이팬+냄비+사은품)
6,32900,http://item.gmarket.co.kr/Item?goodscode=85712...,9900,[영실업]포터리스쿨 도자기스튜디오 LOL서프라이즈 주얼젤리
7,14900,http://item.gmarket.co.kr/Item?goodscode=14260...,4500,[티비제이]TBJ 남녀 여름 티셔츠/팬츠 특가 상품 모음
8,32900,http://item.gmarket.co.kr/Item?goodscode=94895...,9900,올리(Ollie) 여성 여름 샌들/슬리퍼/에스파듀/통굽
9,30000,http://item.gmarket.co.kr/Item?goodscode=63617...,12900,[동원샘물미네마인]동원 샘물 2Lx9병x3팩(총 27병) /생수


In [155]:
ratio = (df['O-price'].astype('int')-df['s-price'].astype('int'))/df['O-price'].astype('int')
ratio = (ratio*100).astype('int').astype('str')+'%'
df['ratio']=ratio
df

,O-price,link,s-price,title,ratio
0,20000,http://item.gmarket.co.kr/Item?goodscode=13005...,12900,[하림]하림 프로라인 치킨너겟 1kg 2봉+치킨너겟 100g(증정),35%
1,19500,http://item.gmarket.co.kr/Item?goodscode=62724...,5900,더다다/원피스형 롱티/신상/루즈/박시/빅사이즈,69%
2,27100,http://item.gmarket.co.kr/Item?goodscode=91202...,16900,CJ 햇반210gx24개/오뚜기밥24개/컵밥10개/컵반,37%
3,65900,http://item.gmarket.co.kr/Item?goodscode=94765...,19900,티셔츠5장9900 원피스/티셔츠/린넨팬츠 세트,69%
4,29000,http://item.gmarket.co.kr/Item?goodscode=14298...,14900,나이키 스우시 라운드 티셔츠,48%
5,100000,http://item.gmarket.co.kr/Item?goodscode=14413...,49900,[풍년]PN풍년 럭키 반값박스 (압력솥+프라이팬+냄비+사은품),50%
6,32900,http://item.gmarket.co.kr/Item?goodscode=85712...,9900,[영실업]포터리스쿨 도자기스튜디오 LOL서프라이즈 주얼젤리,69%
7,14900,http://item.gmarket.co.kr/Item?goodscode=14260...,4500,[티비제이]TBJ 남녀 여름 티셔츠/팬츠 특가 상품 모음,69%
8,32900,http://item.gmarket.co.kr/Item?goodscode=94895...,9900,올리(Ollie) 여성 여름 샌들/슬리퍼/에스파듀/통굽,69%
9,30000,http://item.gmarket.co.kr/Item?goodscode=63617...,12900,[동원샘물미네마인]동원 샘물 2Lx9병x3팩(총 27병) /생수,56%


In [179]:
driver.quit()

In [3]:
url = 'https://hskimim8855.wordpress.com/'
driver = webdriver.Chrome()
driver.get(url)

In [9]:
links =[]
articles = driver.find_elements_by_css_selector('#main > article')
for article in articles:
    links.append(
        article.find_element_by_css_selector('.entry-title > a').get_attribute("href")
    )
len(articles)

1

In [10]:
#링크로 이동
driver.get(links[0])

In [11]:
#스크롤 내리기
script = 'window.scrollTo(0,10000);'
driver.execute_script(script)

In [12]:
#댓글창 클릭
driver.find_element_by_css_selector('#comment-form-comment').click()

In [14]:
#댓글 내용 작성
driver.find_element_by_css_selector('#comment').send_keys('hello world!')

In [15]:
#스크롤 내리기
script = 'window.scrollTo(0,10000);'
driver.execute_script(script)

In [ ]:
#이메일과 이름 데이터 넣기
import time
time.sleep(3)
driver.find_element_by_css_selector('#email').clear()
driver.find_element_by_css_selector('#email').send_keys('hskimim8855@gmail.com')
time.sleep(3)
driver.find_element_by_css_selector('#author').clear()
driver.find_element_by_css_selector('#author').send_keys('hskimim')

In [170]:
#댓글 작성 버튼 클릭
driver.find_element_by_css_selector('#comment-submit').click()

In [32]:
def write_comment(driver,link,msg,email,name):
    driver.get(link)
    script = 'window.scrollTo(0,10000);'
    driver.execute_script(script)
    driver.find_element_by_css_selector('#comment-form-comment').click()
    driver.find_element_by_css_selector('#comment').send_keys(msg)
    script = 'window.scrollTo(0,10000);'
    driver.execute_script(script)
    driver.find_element_by_css_selector('#email').clear()
    driver.find_element_by_css_selector('#email').send_keys(email)
    script = 'window.scrollTo(0,10000);'
    driver.execute_script(script)
    driver.find_element_by_css_selector('#author').clear()
    driver.find_element_by_css_selector('#author').send_keys(name)

In [19]:
links

['https://hskimim8855.wordpress.com/2018/06/21/the-journey-begins/']

In [ ]:
driver = webdriver.Chrome()
write_comment(driver,links[0],'hello world!','hskimim8855@gmail.com','hskimim')
driver.quit()

In [34]:
users = [
    {'email' : "hskimim8855@gmail.com",'name':'kim'},
    {'email' : "helloworld.com",'name':'lee'},
]
msgs = ['helloworld','life is too short']

driver = webdriver.Chrome()

for user in users:
    for msg in msgs:
        for link in links:
            print(user['email'],msg,end = ' - ')
            try:
                write_comment(driver,link,msg,user['email'],user['name'])
                print('success')
            except:
                print('failure')
            time.sleep(1)

{'email': 'hskimim8855@gmail.com', 'name': 'kim'} helloworld https://hskimim8855.wordpress.com/2018/06/21/the-journey-begins/
{'email': 'hskimim8855@gmail.com', 'name': 'kim'} life is too short https://hskimim8855.wordpress.com/2018/06/21/the-journey-begins/
{'email': 'helloworld.com', 'name': 'lee'} helloworld https://hskimim8855.wordpress.com/2018/06/21/the-journey-begins/
{'email': 'helloworld.com', 'name': 'lee'} life is too short https://hskimim8855.wordpress.com/2018/06/21/the-journey-begins/


#### 네이버 메일 크롤링
- 네이버 페이로 이동
- 아이디 패스워드로 로그인
- 전체 메일수, 안읽은 메일 수
- 

In [64]:
from selenium import webdriver

import pickle

main_url = "http://naver.com"

mail_url = "http://mail.naver.com"

driver = webdriver.Chrome()

driver.get(main_url)

In [68]:
with open ('pw.plk','wb') as f:
    pickle.dump('*******',f)

In [87]:
pw = pickle.load(open("pw.plk", "rb")) # pw 변수에 본인의 패스워드 입력
# driver.find_element_by_css_selector(".lg_global > .lg_global_btn").click()
#id
driver.find_element_by_css_selector( ".input_row > .input_box > #id" ).clear()
driver.find_element_by_css_selector( ".input_row > .input_box > #id" ).send_keys( "hskimim8855" )
#pw
driver.find_element_by_css_selector( "#pw" ).clear()
driver.find_element_by_css_selector( "#pw" ).send_keys( pw )
#login button
driver.find_element_by_css_selector("input.btn_global").click()

In [88]:
#mail url 으로 접속
driver.get(mail_url)

In [ ]:
total_mail = driver.find_element_by_css_selector("#headUnreadNum").text
total_mail = int(total_mail)
total_mail

In [ ]:
titles = driver.find_elements_by_css_selector(".mail_title")
names = driver.find_elements_by_css_selector(".name")
for idx, title in enumerate(titles):
    print(idx+1, names[idx].text, titles[idx].text)

In [ ]:
logout_url = driver.find_element_by_css_selector(".gnb_btn_login").get_attribute("href")
driver.get(logout_url)

In [ ]:
driver.close()